# What is the best way to define a game confused


passResult: Dropback outcome of the play (C: Complete pass, I: Incomplete pass, S: Quarterback sack, IN: Intercepted pass, R: Scramble, text)

passLength: The distance beyond the LOS that the ball traveled not including yards into the endzone. If thrown behind LOS, the value is negative. (numeric)

yardsGained: Net yards gained by the offense, including penalty yardage (numeric)

expectedPointsAdded: Expected points added on this play (numeric)

# 1st Plan

Systematic strategy to label plays as "confusing" or not from the defense’s perspective

A scoring system using several independent signals from the data. Each play will be assigned a confusion_score, and we’ll threshold that score to assign a binary label:

Confused = 1 if confusion_score ≥ X (e.g. 3 or 4)

Not Confused = 0 otherwise

| Indicator                          | Points | Description                                                                                  | Data Source           |
| ---------------------------------- | ------ | -------------------------------------------------------------------------------------------- | --------------------- |
| Wide Open Receiver                 | +2     | No defender within 5 yards of the receiver at pass arrival                                   | tracking, player_play |
| Coverage Misassignment             | +1     | Coverage assignment doesn't match tracking proximity                                         | player_play, tracking |
| Multiple Defenders Shift Post-Snap | +1     | > 3 defenders had shiftSinceLineset == True                                                  | player_play           |
| Pre-Snap Motion Confuses Defense   | +1     | Offense uses pre-snap motion; defenders also move (motionSinceLineset)                       | player_play, tracking |
| Erratic Defender Directions        | +1     | High standard deviation in dir of defensive players 0.5s after snap                          | tracking              |
| Pass Coverage Breakdown            | +1     | Zone assigned but defender leaves zone (based on pff_passCoverage vs location at pass catch) | player_play, tracking |
| High EPA for Offense on Play       | +1     | expectedPointsAdded > 4                                                                      | play                  |
| High Motion Count on Defense       | +1     | inMotionAtBallSnap == True for > 4 defenders                                                 | player_play           |


# 2nd Plan

LLM on `playDescription` to detect “chaotic,” “blown,” or “misaligned” plays based on wording. and try to identify id a game is confused or not and take that to train our model.